In [1]:
import torch 
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

In [3]:
def flash_attention(Q, K, V):
    # Step 1: Compute scaled dot-product QK^T in chunks
    chunk_size = 64  # Example chunk size, tune this based on your GPU's memory capacity
    num_chunks = Q.size(1) // chunk_size

    outputs = []
    for i in range(num_chunks):
        start = i * chunk_size
        end = start + chunk_size
        # Compute QK^T for the current chunk
        chunk_attention = torch.matmul(Q[:, start:end, :], K.transpose(-2, -1))
        # Apply softmax and scale (adjust for numerical stability)
        chunk_attention = F.softmax(chunk_attention / torch.sqrt(Q.size(-1)), dim=-1)
        # Store output of the current chunk
        outputs.append(torch.matmul(chunk_attention, V[:, start:end, :]))

    return torch.cat(outputs, dim=1)

# Example input tensors (Q, K, V)
Q = torch.randn(2, 256, 64)  # Batch of 2, sequence length of 256, embedding size of 64
K = Q.clone()
V = Q.clone()

# Perform Flash Attention
# output = flash_attention(Q, K, V)